# IFN647 Week 5 Workshop

Importing the relevant libraries

In [33]:
import string 
from stemming.porter2 import stem
import os

## Task 1 
For the given two XML documents (you can download them from week 5 workshop and then save them in a folder, e.g. 'data'), design a python function **index_docs()** to index them (please remove stop words and index stems only). The returned index should be a dictionary {term: {docid1: freq1, docid2: freq2}}

In [94]:
def index_docs(input_paths, stop_words):
    word_count = 0
    docid = ''
    index = {}

    for path in input_paths:               
        my_file = open(path, 'r')

        stopwords_file = open(stop_words, 'r')
        stop_words_list = stopwords_file.readlines()
        stopwords_file.close()

        stop_words_list = stop_words_list[0].split(',')


        start_end = False
        parsed_text = []
        

        file_ = my_file.readlines()

        for line in file_:
            line = line.strip()

            if line.startswith('<text>'):
                start_end = True
            if line.startswith('<newsitem '):
                for part in line.split():
                    if part.startswith('itemid='):
                        docid = part.split('=')[1].split('/')[0]
                        docid = docid.replace('"', '')
            elif line.startswith('<p>'):
                line = line.replace('<p>', '').replace('</p>', '')
                line = line.translate(str.maketrans('', '', string.punctuation))
                line = line.replace('quot', '')
            elif line.startswith('</text>'):
                start_end = False
            if start_end:
                parsed_text.append(line)
        
        split_text = []

        for line in parsed_text:
            for word in line.split(): 
                word_count += 1

                if word.lower() not in stop_words_list and not word.isdigit():
                    word = word.lower()
                    split_text.append(stem(word))
    
        split_text.remove('<text>')

        # print(split_text)
        for word in split_text:
            # word_count = 0
            if word not in index:
                word_count = 1
            else:
                word_count += 1

            index[word] = {docid: word_count}
        
        # for word in split_text:
        
        my_file.close()
    return index

In [95]:
index = index_docs(['w5_data/data/6146.xml', 'w5_data/data/741299newsML.xml'], 'w5_data/common-english-words.txt')

In [123]:
index

{'argentin': {'6146': 1},
 'bond': {'6146': 1},
 'slight': {'6146': 2},
 'higher': {'6146': 1},
 'small': {'6146': 1},
 'technic': {'6146': 3},
 'bounc': {'6146': 4},
 'wednesday': {'6146': 1},
 'amid': {'6146': 1},
 'low': {'6146': 1},
 'volum': {'6146': 1},
 'trader': {'6146': 2},
 'larg': {'6146': 1},
 'foreign': {'6146': 1},
 'bank': {'6146': 1},
 'open': {'741299': 2},
 'expect': {'6146': 2},
 'price': {'6146': 1},
 'chang': {'6146': 1},
 'much': {'6146': 1},
 'dure': {'741299': 8},
 'session': {'6146': 1},
 'marketmov': {'6146': 1},
 'news': {'6146': 1},
 'percent': {'6146': 1},
 'dollardenomin': {'6146': 1},
 'bocon': {'6146': 1},
 'prevision': {'6146': 1},
 'due': {'6146': 2},
 'rose': {'6146': 3},
 'argentina': {'6146': 2},
 'frb': {'6146': 1},
 'general': {'6146': 1},
 'uncertainti': {'6146': 1},
 'point': {'6146': 1},
 'event': {'6146': 1},
 'market': {'6146': 1},
 'wait': {'6146': 1},
 'includ': {'6146': 1},
 'passag': {'6146': 1},
 'govern': {'6146': 1},
 'new': {'6146': 1

## Task 2
Design a python function **doc_at_a_time(I, Q)** where index I is a dictionary of term:dictionary of (itemid:freq), which returns a dictionary of docId:relevance for the given query Q (a term:freq dictionary)

In [96]:
def likelihood_IR(I, Q):
    L = {}
    R = {}
    D_len = {}

    for list in I.items():
        for id in list[1].items():
            R[id[0]] = 1
            D_len[id[0]] = 0.5

            if list[0] in Q:
                L[list[0]] = I[list[0]]
    
    for q_term in Q.items():
        if not(q_term[0] in L):
            L[q_term[0]] = {}
    
    for list in I.items():
        for id in list[1].items():
            D_len[id[0]] = D_len[id[0]] + id[1]

    for (d, sd) in R.items():
        for (term, f) in L.items():
            if not (d in f):
                f[d] = 0
            sd = sd * (f[d] / D_len[d])
        R[d] = sd
    return R

In [112]:
index_test = index_docs(['w5_data/data/741299newsML.xml'], 'w5_data/common-english-words.txt')

In [130]:
def doc_at_a_time(I, Q):
    IR_result = likelihood_IR(I, Q)
    y = sorted(IR_result.items(), key=lambda x: x[1], reverse=True)

    print('Likelihood Query Results -------------')
    
    for (id, w) in y: 
        print(f'Document ID: {id} and relevance weight: {w}') 
    
    return y

In [131]:
index_test = index_docs(['w5_data/data/6146.xml', 'w5_data/data/741299newsML.xml'], 'w5_data/common-english-words.txt')
doc_at_a_time(index_test, {'argentin': 1})

Likelihood Query Results -------------
Document ID: 6146 and relevance weight: 0.012422360248447204
Document ID: 741299 and relevance weight: 0.0


[('6146', 0.012422360248447204), ('741299', 0.0)]

## Task 3
Design a python function **term_at_a_time(I, Q)**, where index I is a dictionary of a term:dictionary of (itemId:freq), which returns a dictionary of docId:relevance for the given query Q (a term:freq dictionary)

In [151]:
def term_at_a_time(I, Q):
    a = {}
    l = []
    r = []

    k = 2
    g = 1
    f_score = 1

    for term in Q.items():
        if term[0] in I:
            a[term[0]] = I[term[0]]
    
    for li in l:
        try: 
            while True:
                d, score = next(li)
                a[d] += g * f_score
        except StopIteration:
            pass

    for d, score in a.items():
        r.append((d, score))
        if len(r) > k:
            r.pop((d, score))

    return sorted(r, reverse = True)

print(term_at_a_time(index_test, {'belgian': 1}))

[('belgian', {'741299': 2})]


## Task 4 
Design a python main program to call the above three functions for a query, e/g/, Query = {'formula' : 1, 'one' : 1}

In [ ]:
def main():
    index = index_docs(['w5_data/data/6146.xml', ], 'w5_data/common-english-words.txt')
    doc_at_time = doc_at_a_time(index, {'argentin': 1})
    term_at_time = term_at_a_time(index, {'belgian': 1})

    print('Document at a Time Result', doc_at_time)
    print('Term at a Time Result', term_at_time)

main()

Likelihood Query Results -------------
Document ID: 6146 and relevance weight: 0.012121212121212121
Document at a Time Result [('6146', 0.012121212121212121)]
Term at a Time Result []
